# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [2]:
# Connect to SQLite database.
conn = sqlite3.connect('data/prod_sample.db')
conn

In [3]:
# List all the tables in the database.
query = "SELECT name from sqlite_master where type= 'table'"
table_list = pd.read_sql(query,conn)
table_list.head()

,name
0,stock_description
1,online_retail_history


# Read data from the `online_retail_history` table

In [4]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = "SELECT * from online_retail_history"

# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history = pd.read_sql(query,conn)

# Preview the first five rows of the data.
online_retail_history.head(n = 5)

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [5]:
# Get the shape of the data.
online_retail_history.shape

(15321, 8)

# Read data from the `stock_description` table

In [6]:
# Write the query to be executed that selects everything from the stock_description table.
query = "SELECT * from stock_description"

# Use the read_sql function in pandas to read a query into a DataFrame.
stock_description = pd.read_sql(query,conn)

# Preview the first five rows of the data.
stock_description.head(n = 5)

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [7]:
# Get the shape of the data.
stock_description.shape

(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [8]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.
query = '''Select orh.*, sd.Description from online_retail_history as orh 
inner join stock_description as sd on orh.StockCode = sd.StockCode'''






# Use the read_sql function in pandas to read a query into a DataFrame.
online_retail_history_and_stock_description_agg = pd.read_sql(query,conn)


# Preview the first five rows of the data.
online_retail_history_and_stock_description_agg.head(n = 5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


In [21]:
# Get the shape of the data.

online_retail_history_and_stock_description_agg.shape

(17032, 9)

# Identify and fix corrupt or unusable data

In [22]:
# Check the value counts of the "Description" field.

online_retail_history_and_stock_description_agg.Description.value_counts()

CREAM HANGING HEART T-LIGHT HOLDER    2174
JUMBO BAG RED RETROSPOT               1960
?                                     1711
REGENCY CAKESTAND 3 TIER              1711
PARTY BUNTING                         1615
LUNCH BAG RED RETROSPOT               1421
ASSORTED COLOUR BIRD ORNAMENT         1405
POPCORN HOLDER                        1329
LUNCH BAG  BLACK SKULL.               1271
SET OF 3 CAKE TINS PANTRY DESIGN      1257
PACK OF 72 RETROSPOT CAKE CASES       1178
Name: Description, dtype: int64

In [24]:
# Remove rows where "Description" is just a question mark (?).
online_retail_and_stock_cleaned = \
online_retail_history_and_stock_description_agg[
    online_retail_history_and_stock_description_agg.Description != '?']



# Preview the first five rows of the data.
online_retail_and_stock_cleaned.head(n = 5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Identify and remove duplicates

In [26]:
# Identify all duplicated data.
duplicated_data = \
online_retail_and_stock_cleaned[online_retail_and_stock_cleaned.duplicated(keep = False)]




In [27]:
# Print the duplicated data.
print("No. of rows with Duplicated Data: ", duplicated_data.shape[0])


No. of rows with Duplicated Data:  223


In [28]:
# Remove the duplicated data.
online_retail_and_stock_cleaned = \
online_retail_and_stock_cleaned[~online_retail_and_stock_cleaned.duplicated()]






# Preview the first five rows of the data.

online_retail_and_stock_cleaned.head(n = 5)

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Correct date formats

In [29]:
# Get the data types for every column in the DataFrame.
online_retail_and_stock_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15206 non-null  object 
 1   StockCode    15206 non-null  object 
 2   Quantity     15206 non-null  int64  
 3   InvoiceDate  15206 non-null  object 
 4   Price        15194 non-null  float64
 5   CustomerID   12435 non-null  object 
 6   Country      15206 non-null  object 
 7   TotalAmount  15194 non-null  float64
 8   Description  15206 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [30]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.

online_retail_and_stock_cleaned['InvoiceDate'] = \
pd.to_datetime(online_retail_and_stock_cleaned['InvoiceDate'], format = '%Y-%m-%d')


In [31]:
# Get the data types for every column in the converted DataFrame.
online_retail_and_stock_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 15206 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15206 non-null  object        
 1   StockCode    15206 non-null  object        
 2   Quantity     15206 non-null  int64         
 3   InvoiceDate  15206 non-null  datetime64[ns]
 4   Price        15194 non-null  float64       
 5   CustomerID   12435 non-null  object        
 6   Country      15206 non-null  object        
 7   TotalAmount  15194 non-null  float64       
 8   Description  15206 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [32]:
# Preview the first five rows of the data.
online_retail_and_stock_cleaned.head(n = 5)


,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount,Description
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08,ASSORTED COLOUR BIRD ORNAMENT
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30,CREAM HANGING HEART T-LIGHT HOLDER
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50,LUNCH BAG RED RETROSPOT


# Load the dataset into a pickle file

In [33]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.

online_retail_and_stock_cleaned.to_pickle('online_history_cleaned.pickle')

In [34]:
# Close any connections to the database.
conn.close()
